In [1]:
# mentioning datatypes for better documentation
from typing import Dict, List
from datasets import Dataset, load_dataset, disable_caching
disable_caching() ## disable huggingface cache
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch
from torch.utils.data import Dataset
from IPython.display import Markdown

In [2]:
# Dataset Preparation
dataset = load_dataset("MBZUAI/LaMini-instruction" , split = 'train') 
small_dataset = dataset.select([i for i in range(200)])
print(small_dataset)
print(small_dataset[0])

# creating templates
prompt_template = """Below is an instruction that describes a task. Write a response that appropriately completes the request. Instruction: {instruction}\n Response:"""
answer_template = """{response}"""

# creating function to add keys in the dictionary for prompt, answer and whole text
def _add_text(rec):
    instruction = rec["instruction"]
    response = rec["response"] 
    # check if both exists, else raise error   
    if not instruction:
        raise ValueError(f"Expected an instruction in: {rec}")
    if not response:
        raise ValueError(f"Expected a response in: {rec}")
    rec["prompt"] = prompt_template.format(instruction=instruction)
    rec["answer"] = answer_template.format(response=response)
    rec["text"] = rec["prompt"] + rec["answer"]
    return rec

# running through all samples
small_dataset = small_dataset.map(_add_text)
print(small_dataset[0])

Dataset({
    features: ['instruction', 'response', 'instruction_source'],
    num_rows: 200
})
{'instruction': 'List 5 reasons why someone should learn to code', 'response': '1. High demand for coding skills in the job market\n2. Increased problem-solving and analytical skills\n3. Ability to develop new products and technologies\n4. Potentially higher earning potential\n5. Opportunity to work remotely and/or freelance', 'instruction_source': 'alpaca'}


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

{'instruction': 'List 5 reasons why someone should learn to code', 'response': '1. High demand for coding skills in the job market\n2. Increased problem-solving and analytical skills\n3. Ability to develop new products and technologies\n4. Potentially higher earning potential\n5. Opportunity to work remotely and/or freelance', 'instruction_source': 'alpaca', 'prompt': 'Below is an instruction that describes a task. Write a response that appropriately completes the request. Instruction: List 5 reasons why someone should learn to code\n Response:', 'answer': '1. High demand for coding skills in the job market\n2. Increased problem-solving and analytical skills\n3. Ability to develop new products and technologies\n4. Potentially higher earning potential\n5. Opportunity to work remotely and/or freelance', 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request. Instruction: List 5 reasons why someone should learn to code\n Response:

In [3]:
from transformers import BitsAndBytesConfig

# loading the tokenizer for dolly model. The tokenizer converts raw text into tokens
model_id = "databricks/dolly-v2-3b"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

#loading the model using AutoModelForCausalLM
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    # use_cache=False,
    device_map="auto", #"balanced",
    quantization_config=BitsAndBytesConfig(load_in_8bit=True),
    torch_dtype=torch.float16
)

# resizes input token embeddings matrix of the model if new_num_tokens != config.vocab_size.
base_model.resize_token_embeddings(len(tokenizer))

Embedding(50280, 2560)

In [4]:
from functools import partial
import copy
from transformers import DataCollatorForSeq2Seq

MAX_LENGTH = 256

# Function to generate token embeddings from text part of batch
def _preprocess_batch(batch: Dict[str, List]):  
    model_inputs = tokenizer(batch["text"], max_length=MAX_LENGTH, truncation=True, padding='max_length')    
    model_inputs["labels"] = copy.deepcopy(model_inputs['input_ids'])
    return model_inputs

_preprocessing_function = partial(_preprocess_batch)

# apply the preprocessing function to each batch in the dataset
encoded_small_dataset = small_dataset.map(
        _preprocessing_function,
        batched=True,
        remove_columns=["instruction", "response", "prompt", "answer"],
)
processed_dataset = encoded_small_dataset.filter(lambda rec: len(rec["input_ids"]) <= MAX_LENGTH)

# splitting dataset
split_dataset = processed_dataset.train_test_split(test_size=14, seed=0)
print(split_dataset)

# takes a list of samples from a Dataset and collate them into a batch, as a dictionary of PyTorch tensors.
data_collator = DataCollatorForSeq2Seq(
        model = base_model, tokenizer=tokenizer, max_length=MAX_LENGTH, pad_to_multiple_of=8, padding='max_length')

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Filter:   0%|          | 0/200 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction_source', 'text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 186
    })
    test: Dataset({
        features: ['instruction_source', 'text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 14
    })
})


In [5]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

LORA_R = 256 # 512
LORA_ALPHA = 512 # 1024
LORA_DROPOUT = 0.05
# Define LoRA Config
lora_config = LoraConfig(
                 r = LORA_R, # the dimension of the low-rank matrices
                 lora_alpha = LORA_ALPHA, # scaling factor for the weight matrices
                 lora_dropout = LORA_DROPOUT, # dropout probability of the LoRA layers
                 bias="none",
                 task_type="CAUSAL_LM",
                 target_modules=["query_key_value"],
)

# Prepare int-8 model for training - utility function that prepares a PyTorch model for int8 quantization training. <https://huggingface.co/docs/peft/task_guides/int8-asr>
model = prepare_model_for_kbit_training(base_model, 8)
# initialize the model with the LoRA framework
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 83,886,080 || all params: 2,858,972,160 || trainable%: 2.9341


**Training using Trainer Class**

In [6]:
# from transformers import TrainingArguments, Trainer
# import bitsandbytes
# # define the training arguments first.
# EPOCHS = 3
# LEARNING_RATE = 1e-4  
# MODEL_SAVE_FOLDER_NAME = "dolly-3b-lora"
# training_args = TrainingArguments(
#                     output_dir=MODEL_SAVE_FOLDER_NAME,
#                     overwrite_output_dir=True,
#                     fp16=True, #converts to float precision 16 using bitsandbytes
#                     per_device_train_batch_size=1,
#                     per_device_eval_batch_size=1,
#                     learning_rate=LEARNING_RATE,
#                     num_train_epochs=EPOCHS,
#                     logging_strategy="epoch",
#                     evaluation_strategy="epoch",
#                     save_strategy="epoch",
# )
# # training the model 
# trainer = Trainer(
#         model=model,
#         tokenizer=tokenizer,
#         args=training_args,
#         train_dataset=split_dataset['train'],
#         eval_dataset=split_dataset["test"],
#         data_collator=data_collator,
# )
# model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
# trainer.train()
# # only saves the incremental 🤗 PEFT weights (adapter_model.bin) that were trained, meaning it is super efficient to store, transfer, and load.
# trainer.model.save_pretrained(MODEL_SAVE_FOLDER_NAME)
# # save the full model and the training arguments
# trainer.save_model(MODEL_SAVE_FOLDER_NAME)
# trainer.model.config.save_pretrained(MODEL_SAVE_FOLDER_NAME)

**Custom Training Loop**

In [11]:
import torch.nn.functional as F

# Example custom loss function
def custom_loss(outputs, labels):
    # Mean Squared Error as an example
    return F.mse_loss(outputs, labels)

from torch.utils.data import DataLoader
from tqdm import tqdm

EPOCHS = 3
LEARNING_RATE = 1e-4  
MODEL_SAVE_FOLDER_NAME = "dolly-3b-lora"

# Define your optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

# Create DataLoader for train and test datasets
train_loader = DataLoader(split_dataset['train'], batch_size=1, shuffle=True)
eval_loader = DataLoader(split_dataset['test'], batch_size=1)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
for epoch in range(EPOCHS):
    model.train()  # Set model to training mode
    train_loss = 0.0
    
    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch + 1}/{EPOCHS}"):
        print("AAAAAA")
        for k, v in batch.items():
            print(f"{k}: {v}")
        print("BBBBBB")
        # Move batch data to device
        inputs = {k: v.to(device) for k, v in batch.items()}

        # Forward pass
        outputs = model(**inputs)
        loss = custom_loss(outputs.logits, inputs["labels"])  # Use your custom loss here

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
    
    print(f"Epoch {epoch + 1} - Training Loss: {train_loss / len(train_loader)}")

    # Evaluation loop
    model.eval()  # Set model to evaluation mode
    eval_loss = 0.0
    with torch.no_grad():
        for batch in tqdm(eval_loader, desc="Evaluating"):
            inputs = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**inputs)
            loss = custom_loss(outputs.logits, inputs["labels"])  # Use the same custom loss
            eval_loss += loss.item()
    
    print(f"Epoch {epoch + 1} - Evaluation Loss: {eval_loss / len(eval_loader)}")

# Save the trained model
model.save_pretrained(MODEL_SAVE_FOLDER_NAME)
tokenizer.save_pretrained(MODEL_SAVE_FOLDER_NAME)


Training Epoch 1/3:   0%|          | 0/186 [00:00<?, ?it/s]

Training Epoch 1/3:   0%|          | 0/186 [00:00<?, ?it/s]


AAAAAA
instruction_source: ['alpaca']
text: ['Below is an instruction that describes a task. Write a response that appropriately completes the request. Instruction: List three characteristics of an extrovert\n Response:1. Outgoing and sociable personality.\n2. Tendency to seek social stimulation and interaction with others.\n3. Draw their energy from interaction with others.']
input_ids: [tensor([30003]), tensor([310]), tensor([271]), tensor([9775]), tensor([326]), tensor([8631]), tensor([247]), tensor([4836]), tensor([15]), tensor([19566]), tensor([247]), tensor([2380]), tensor([326]), tensor([20420]), tensor([29141]), tensor([253]), tensor([2748]), tensor([15]), tensor([41959]), tensor([27]), tensor([5552]), tensor([1264]), tensor([5319]), tensor([273]), tensor([271]), tensor([1021]), tensor([287]), tensor([1748]), tensor([187]), tensor([19371]), tensor([27]), tensor([18]), tensor([15]), tensor([6282]), tensor([5681]), tensor([285]), tensor([10621]), tensor([494]), tensor([13216]), t

AttributeError: 'list' object has no attribute 'to'

In [ ]:
# Function to format the response and filter out the instruction from the response.
def postprocess(response):
    messages = response.split("Response:")
    if not messages:
        raise ValueError("Invalid template for prompt. The template should include the term 'Response:'")
    return "".join(messages[1:])
# Prompt for prediction
inference_prompt = "List 5 reasons why someone should learn to cook"
# Inference pipeline with the fine-tuned model
inf_pipeline =  pipeline('text-generation', model=trainer.model, tokenizer=tokenizer, max_length=256, trust_remote_code=True)
# inf_pipeline =  pipeline('text-generation', model=base_model, tokenizer=tokenizer, max_length=256, trust_remote_code=True)
# Format the prompt using the `prompt_template` and generate response 
response = inf_pipeline(prompt_template.format(instruction=inference_prompt))[0]['generated_text']
# postprocess the response
formatted_response = postprocess(response)
formatted_response